# Тренировка аи модели для RVC
## Без гуи, но с русским языком
 
**Запуск в фоновом режиме:**  
Save Versions - Save Output - Always Save Output... - SAVE и ждем начало тренировки! 
 
**Где лежит модель?:** 
В папке Output в .zip архиве 

# НАШ ТРЕД https://2ch.hk/ai/res/664162.html
 
#### Версия Скрипта: v.1.0  
#### УБРАЛ РЕКЛАМУ И ДАЛ АНОНАМ: https://github.com/hinaichigo-fox/rvc_train_for_anon

**Что можно редактировать?**

НАЗВАНИЕ ТВОЕГО DATASETА DATASET = "ИМЯ-ДАТАСЕТА" заменяем имя-датасета на наш датасет

Параметры модели MODELEPOCH = 300 вместо 300 ставим нужное кол-во эпох

Имя Модели (Замените "model_name") ZIP_NAME = "model_name.zip" вместо model_name ставим нужное имя. на пиндосском без пробелов. Модель переименуется вместе с архивом

**БОЛЬШЕ НИЧЕГО НЕ ТРОГАЙТЕ!!!(если не знаете что делаете)**

### Установка

In [ ]:
# Установка необходимых зависимостей
!apt-get -y install build-essential python3-dev ffmpeg
!pip3 install --upgrade setuptools wheel
!pip3 install --upgrade pip
!pip3 install faiss-cpu==1.7.2 fairseq gradio==3.14.0 ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2
!pip3 install torch torchvision torchaudio

# Создание каталогов для рабочей среды
!mkdir content
!mkdir content/rmvpe-ai
%cd /kaggle/working/content/rmvpe-ai

# Клонирование репозитория rmvpe-ai с выбранным веткой 'v1.1-patch-no-ui'
!git clone -b 'v1.1-patch-no-ui' --single-branch --depth 1 https://github.com/elcolex777/rmvpe-ai.git .

# Установка зависимостей из requirements.txt
!pip install -r requirements.txt

# Установка aria2 для управления загрузками
!apt -y install -qq aria2

In [ ]:
# Загрузка файлов и т.д 
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://bit.ly/3PnwhB6 -d /kaggle/working/content/rmvpe-ai/pretrained_v2 -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://bit.ly/45Oa5FE -d /kaggle/working/content/rmvpe-ai/pretrained_v2 -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://bit.ly/45J4v80 -d /kaggle/working/content/rmvpe-ai/pretrained_v2 -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://bit.ly/45XHOfP -d /kaggle/working/content/rmvpe-ai/pretrained_v2 -o f0G40k.pth

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://bit.ly/3L9mNa8 -d /kaggle/working/content/rmvpe-ai -o hubert_base.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://bit.ly/47Yxi9Y -d /kaggle/working/content/rmvpe-ai -o rmvpe.pt

## **НАЗВАНИЕ ТВОЕГО DATASETА**

In [ ]:
# НАЗВАНИЕ ТВОЕГО DATASETА
DATASET = "ИМЯ-ДАТАСЕТА"

# Создаем каталог "dataset_raw", если его нет
!mkdir -p "dataset_raw"

# Копируем все файлы и данные из директории /kaggle/input/{DATASET}/ в каталог "dataset_raw/"
!cp -R /kaggle/input/{DATASET}/* -t "dataset_raw/"

In [ ]:
# Изменяем текущую рабочую директорию на /kaggle/working/content/rmvpe-ai
%cd /kaggle/working/content/rmvpe-ai

In [ ]:
!mkdir -p logs/mi-test
!touch logs/mi-test/preprocess.log
#!touch logs/mi-test/filelist.txt
#!touch logs/filelist.txt

!touch logs/mi-test/extract_f0_feature.log
#python extract_f0_print.py logs/mi-test $(nproc) pm
#python extract_feature_print.py cpu 1 0 0 logs/braga01 v1

In [ ]:
# @markdown Название модели
MODELNAME = "mi-test"  # @param {type:"string"}
# @markdown Битрейт выборки
BITRATE = 40000  # @param {type:"integer"}
# @markdown Количество потоков
THREADCOUNT = 8  # @param {type:"integer"}

# Запуск скрипта trainset_preprocess_pipeline_print.py с указанными параметрами
!python3 trainset_preprocess_pipeline_print.py /kaggle/working/content/rmvpe-ai/dataset_raw {BITRATE} {THREADCOUNT} logs/{MODELNAME} True

In [ ]:
#!python3 extract_f0_rmvpe.py 2 0 0 "/kaggle/working/content/rmvpe-ai/logs/mi-test" True
#!python3 extract_f0_rmvpe.py 2 1 0 "/kaggle/working/content/rmvpe-ai/logs/mi-test" True

### Тип модели (ALGO) | Стоит RMVPE

In [ ]:
# @markdown Название модели
MODELNAME = "mi-test"  # @param {type:"string"}
# @markdown Количество потоков
THREADCOUNT = 8  # @param {type:"integer"}
# @markdown Алгоритм извлечения высоты тона
ALGO = "rmvpe"  # @param {type:"string"} pm|harvest|crepe|rmvpe

# Запуск скрипта extract_f0_print.py с указанными параметрами
!python3 extract_f0_print.py logs/{MODELNAME} {THREADCOUNT} {ALGO}

# Запуск скрипта extract_feature_print.py с указанными параметрами
!python3 extract_feature_print.py cpu 1 0 0 logs/{MODELNAME} v2

In [ ]:
# @markdown Название модели
MODELNAME = "mi-test"  # @param {type:"string"}
# @markdown Частота дискретизации модели
MODELSAMPLE = "40k"  # @param {type:"string"}

# Запуск скрипта create_filelist_print.py с указанными параметрами
!python3 create_filelist_print.py {MODELNAME} v2 True {MODELSAMPLE} 0

In [ ]:
# @markdown Название модели
MODELNAME = "mi-test"  # @param {type:"string"}

# Запуск скрипта train_index_print.py с указанным названием модели
!python3 train_index_print.py {MODELNAME} v2

### Параметры модели

In [ ]:
import threading
import os
import time

# @markdown Название модели
MODELNAME = "mi-test"  # @param {type:"string"}
# @markdown Используемая GPU (Значение 0,1 две видокарты от GPU T4 X2)
USEGPU = "1"  # @param {type:"string"}
# @markdown Размер пакета (batch size)
BATCHSIZE = 7  # @param {type:"integer"}
# @markdown КОЛ-ВО ЭПОХ
MODELEPOCH = 300  # @param {type:"integer"}
# @markdown Интервал сохранения модели (через сколько эпох)
EPOCHSAVE = 50  # @param {type:"integer"}
# @markdown Частота дискретизации модели
MODELSAMPLE = "40k"  # @param {type:"string"}
# @markdown Кэшировать данные обучающего набора (1 - Да, 0 - Нет)
CACHEDATA = 0  # @param {type:"integer"}
# @markdown Сохранять только последний файл весов (1 - Да, 0 - Нет)
ONLYLATEST = 0  # @param {type:"integer"} 

def train_model():
    os.system(f"python3 train_nsf_sim_cache_sid_load_pretrain.py -e {MODELNAME} -sr {MODELSAMPLE} -f0 1 -bs {BATCHSIZE} -g {USEGPU} -te {MODELEPOCH} -se {EPOCHSAVE} -pg pretrained_v2/f0G{MODELSAMPLE}.pth -pd pretrained_v2/f0D{MODELSAMPLE}.pth -l {ONLYLATEST} -c {CACHEDATA} -sw 1 -v v2")

def tail_log():
    log_file = "/kaggle/working/content/rmvpe-ai/logs/" + MODELNAME + "/train.log"
    last_epoch = None
    last_position = 0
    training_started = False
    while True:
        if os.path.exists(log_file):
            with open(log_file, 'r') as file:
                file.seek(last_position)
                lines = file.readlines()
                last_position = file.tell()
                for line in lines:
                    if "Epoch:" in line:
                        current_epoch = int(line.split("Epoch:")[1].split()[0])
                        if not training_started and current_epoch == 1:
                            print("НАЧАЛАСЬ ТРЕНИРОВКА ГОЛОСОВОЙ МОДЕЛИ")
                            training_started = True
                        if current_epoch != last_epoch:
                            print(line.strip().replace(f"Epoch: {current_epoch}", f"Epoch: {current_epoch} \ {MODELEPOCH}"))
                            last_epoch = current_epoch
                        if current_epoch % EPOCHSAVE == 0:
                            print(f"Сохранено {current_epoch} Эпох")
                        if current_epoch == MODELEPOCH:
                            check_model_file()
        time.sleep(10)  # Проверяем файл каждые 10 секунд

def check_model_file():
    pth_file = f"/kaggle/working/content/rmvpe-ai/weights/{MODELNAME}.pth"
    
    if os.path.exists(pth_file):
        print(f'Модель "{MODELNAME}" Готова!')
        raise SystemExit("Завершение выполнения ячейки")
    else:
        print(f'Файл модели "{MODELNAME}" не найден.')

# Create threads
train_thread = threading.Thread(target=train_model)
tail_thread = threading.Thread(target=tail_log)

# Start threads
train_thread.start()
tail_thread.start()

# Join threads
train_thread.join()
tail_thread.join()

## **Имя Модели (Замените "model_name")**

In [ ]:
import os
import zipfile
import glob

# Задайте имя ZIP-архива здесь:
ZIP_NAME = "model_name.zip"

def check_and_zip_files(model_name, zip_name=ZIP_NAME):
    base_name = os.path.splitext(zip_name)[0]  # Убираем расширение .zip
    
    # Поиск файла .index с учетом возможного различного числа после IVF
    index_files = glob.glob(f"/kaggle/working/content/rmvpe-ai/logs/{model_name}/added_IVF*_Flat_nprobe_1_{model_name}_v2.index")
    if not index_files:
        return False
    index_file_path = index_files[0]
    index_file_name = os.path.basename(index_file_path).replace(model_name, base_name)
    
    pth_file = f"/kaggle/working/content/rmvpe-ai/weights/{model_name}.pth"
    
    if os.path.exists(index_file_path) and os.path.exists(pth_file):
        with zipfile.ZipFile(f"/kaggle/working/{zip_name}", 'w') as zipf:
            zipf.write(index_file_path, index_file_name)
            zipf.write(pth_file, f"{base_name}.pth")
        return True
    return False

# После завершения тренировки:
if check_and_zip_files(MODELNAME):
    print(f"Файлы модели упакованы в {ZIP_NAME}")
else:
    print("Не удалось найти файлы модели.")